In [1]:
import numpy as np
import qilisdk
import qutip as qt
import matplotlib.pyplot as plt

In [5]:
from qilisdk.analog import Hamiltonian, X, Y, Z, I

# Build a Hamiltonian expression using Pauli operators
H_expr = (Z(0) + X(0)) * (Z(0) - X(0))
H_expr = Z(0)*Z(1)*I(0) 
print("Hamiltonian Expression:", H_expr)

# Iterate over Hamiltonian terms
for coeff, ops in H_expr:
    print("Coefficient:", coeff, "Operators:", ops)

print("-"*10)
# export hamiltonians to matrices
print("Sparse Matrix from Hamiltonian: \n", H_expr.to_matrix()) # sparse matrix  
# the returned matrix is sparse by default to get the dense representation call .toarray()


Hamiltonian Expression: Z(0) Z(1)
Coefficient: (1+0j) Operators: [Z(0), Z(1)]
----------
Sparse Matrix from Hamiltonian: 
 <Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 4 stored elements and shape (4, 4)>
  Coords	Values
  (0, 0)	(1+0j)
  (1, 1)	(-1+0j)
  (2, 2)	(-1+0j)
  (3, 3)	(1+0j)


In [105]:
class Hamiltonian():
    
    def __init__(self,size):
        self.size = size
        self.H = 0
    
    def add_local_field(self, qubit, field, weight=1.):

        assert qubit <= self.size-1, f"Index {qubit} out of range. Size of the system is {self.size}"
        
        self.H += weight*field(qubit)

    def add_ZZ_term(self, qubit1, qubit2, weight=1.):

        assert qubit1<= self.size-1, f"Index {qubit1} out of range. Size of the system is {self.size}"
        assert qubit2 <= self.size-1, f"Index {qubit2} out of range. Size of the system is {self.size}"
        assert qubit1 != qubit2,    f"Qubits 1 and 2 are the same, cannot apply self-interaction term"

        self.H += weight*Z(qubit1)*Z(qubit2)
        
              


In [ ]:
def create_random_hamiltonian(Nqubits, min_weight, max_weight):
    H_out = Hamiltonian(Nqubits)
    single_x_weights = np.random.uniform(min_weight, max_weight, size=Nqubits)
    single_z_weights = np.random.uniform(min_weight, max_weight, size=Nqubits)

    interaction_weights = np.random.uniform(min_weight, max_weight, size=int(Nqubits*(Nqubits-1)/2))
    interaction_counter = 0
    for i in range(Nqubits):
        H_out.add_local_field(i, X, single_x_weights[i])
        H_out.add_local_field(i, Z, single_z_weights[i])

        for j in range(i+1, Nqubits):
            H_out.add_ZZ_term(i, j, interaction_weights[interaction_counter])
            interaction_counter+=1
    
    return H_out

In [129]:
H_test = create_random_hamiltonian(1, -1.0, 1.0)
